# Base Distillation

the baseline for the distillation matrics, including:
- a converged teacher ResNet-56 on CIFAR-100
- a converged student ResNet-20 on CIFAR-100
- a converged student ResNet-20 with Knowledge Distillation on CIFAR-100

the accuracy of these three model will served as the benchmark for the rest experient

In [2]:
from utils.download_cifar100 import download_cifar100

# 先加载数据集 CIFAR-100
trainset, testset, trainloader, testloader = download_cifar100()

开始下载CIFAR-100数据集...
下载训练集中...
下载测试集中...
下载完成！
训练集样本数: 50000
测试集样本数: 10000
类别数: 100


In [ ]:
from utils.base_model import (
    train_teacher_model,
    train_student_vanilla,
    train_student_with_distillation,
)


def run_all_benchmarks():
    """运行所有基准模型训练"""
    print("=" * 60)
    print("开始训练所有基准模型...")
    print("=" * 60)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用设备: {device}")

    results = {}

    # 1. 训练教师模型
    print("\n1. 训练教师模型 ResNet-56")
    teacher_model, teacher_losses, teacher_accuracies = train_teacher_model()
    results["teacher"] = {
        "model": teacher_model,
        "best_acc": max(teacher_accuracies) if teacher_accuracies else 0,
    }

    # 2. 训练学生模型（无蒸馏）
    print("\n2. 训练学生模型 ResNet-20（无蒸馏）")
    student_vanilla, vanilla_acc = train_student_vanilla()
    results["student_vanilla"] = {"model": student_vanilla, "best_acc": vanilla_acc}

    # 3. 训练学生模型（带蒸馏）
    print("\n3. 训练学生模型 ResNet-20（带知识蒸馏）")
    student_kd, kd_acc = train_student_with_distillation(teacher_model)
    results["student_kd"] = {"model": student_kd, "best_acc": kd_acc}

    # 打印最终结果比较
    print("\n" + "=" * 60)
    print("基准模型训练完成！结果汇总:")
    print("=" * 60)
    print(f"教师模型 (ResNet-56): {results['teacher']['best_acc']:.2f}%")
    print(f"学生模型无蒸馏 (ResNet-20): {results['student_vanilla']['best_acc']:.2f}%")
    print(f"学生模型带蒸馏 (ResNet-20): {results['student_kd']['best_acc']:.2f}%")

    # 计算提升
    improvement = (
        results["student_kd"]["best_acc"] - results["student_vanilla"]["best_acc"]
    )
    print(f"知识蒸馏带来的准确率提升: {improvement:+.2f}%")

    return results


def plot_comparison(results):
    """绘制结果比较图"""
    import matplotlib.pyplot as plt

    models = [
        "Teacher\nResNet-56",
        "Student\nNo KD\nResNet-20",
        "Student\nWith KD\nResNet-20",
    ]
    accuracies = [
        results["teacher"]["best_acc"],
        results["student_vanilla"]["best_acc"],
        results["student_kd"]["best_acc"],
    ]

    plt.figure(figsize=(10, 6))
    bars = plt.bar(models, accuracies, color=["#FF6B6B", "#4ECDC4", "#45B7D1"])
    plt.ylabel("Test Accuracy (%)")
    plt.title("Knowledge Distillation Benchmark Results on CIFAR-100")
    plt.ylim(0, 100)

    # 在柱状图上显示数值
    for bar, acc in zip(bars, accuracies):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 1,
            f"{acc:.2f}%",
            ha="center",
            va="bottom",
        )

    plt.tight_layout()
    plt.savefig("benchmark_results.png", dpi=150, bbox_inches="tight")
    plt.show()


# 运行所有基准训练
results = run_all_benchmarks()

# 绘制比较图
plot_comparison(results)

print("\n🎉 所有基准模型训练完成！")
print("📁 模型已保存为:")
print("   - teacher_resnet56_best.pth")
print("   - student_vanilla_resnet20_best.pth")
print("   - student_kd_resnet20_best.pth")
print("📊 结果图已保存为: benchmark_results.png")

开始训练所有基准模型...
使用设备: cuda

1. 训练教师模型 ResNet-56
🚀 开始训练教师模型 ResNet-56...


Epoch 1/200:   0%|          | 0/391 [00:00<?, ?it/s]

In [2]:
from utils.res_net import ResNet56, ResNet20
from utils.base_model import test_model
from utils.cifar100 import CIFAR100Data
import torch


def diagnose_models():
    """诊断模型性能问题"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 加载已保存的模型
    teacher = ResNet56(num_classes=100).to(device)
    student_vanilla = ResNet20(num_classes=100).to(device)
    student_kd = ResNet20(num_classes=100).to(device)

    # 加载权重
    teacher.load_state_dict(torch.load("../model_weights/teacher_resnet56_best.pth"))
    student_vanilla.load_state_dict(
        torch.load("../model_weights/student_vanilla_resnet20_best.pth")
    )
    student_kd.load_state_dict(
        torch.load("../model_weights/student_kd_resnet20_best.pth")
    )

    # 测试准确率
    data_manager = CIFAR100Data(batch_size=100)
    _, testloader = data_manager.get_dataloaders()

    teacher_acc = test_model(teacher, testloader, device)
    vanilla_acc = test_model(student_vanilla, testloader, device)
    kd_acc = test_model(student_kd, testloader, device)

    print(f"教师模型 (ResNet-56): {teacher_acc:.2f}%")
    print(f"学生模型无蒸馏 (ResNet-20): {vanilla_acc:.2f}%")
    print(f"学生模型带蒸馏 (ResNet-20): {kd_acc:.2f}%")

    # 检查模型大小和参数
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"\n模型参数数量:")
    print(f"ResNet-56: {count_parameters(teacher):,}")
    print(f"ResNet-20: {count_parameters(student_vanilla):,}")


# 运行诊断
diagnose_models()

教师模型 (ResNet-56): 77.54%
学生模型无蒸馏 (ResNet-20): 76.66%
学生模型带蒸馏 (ResNet-20): 77.85%

模型参数数量:
ResNet-56: 55,115,172
ResNet-20: 17,490,852


In [1]:
from utils.base_model import train_micro_model

micro_model, best_acc = train_micro_model()

MicroResNet Epoch 1/200: 100%|██████████| 391/391 [00:08<00:00, 48.46it/s, Loss=3.820, Acc=7.69%] 


MicroResNet Epoch 1: Test Accuracy = 9.98%
✅ 新的最佳准确率: 9.98%


MicroResNet Epoch 2/200: 100%|██████████| 391/391 [00:08<00:00, 48.49it/s, Loss=3.419, Acc=12.13%] 


MicroResNet Epoch 2: Test Accuracy = 11.52%
✅ 新的最佳准确率: 11.52%


MicroResNet Epoch 3/200: 100%|██████████| 391/391 [00:08<00:00, 48.07it/s, Loss=3.356, Acc=15.44%] 


...

MicroResNet Epoch 198: Test Accuracy = 53.15%


MicroResNet Epoch 199/200: 100%|██████████| 391/391 [00:07<00:00, 49.60it/s, Loss=1.837, Acc=53.57%] 


MicroResNet Epoch 199: Test Accuracy = 52.67%


MicroResNet Epoch 200/200: 100%|██████████| 391/391 [00:08<00:00, 48.60it/s, Loss=1.426, Acc=53.73%] 


MicroResNet Epoch 200: Test Accuracy = 52.85%
🎉 超小型ResNet训练完成! 最佳准确率: 53.24%


In [1]:
from utils.base_model import train_micro_student_with_distillation
from utils.res_net import ResNet20
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = ResNet20(num_classes=100).to(device)
teacher_model.load_state_dict(
    torch.load("../model_weights/student_vanilla_resnet20_best.pth")
)

micro_model_kd, best_acc_kd = train_micro_student_with_distillation(teacher_model)

🚀 开始训练学生模型 MicroResNet（带知识蒸馏）...


Student KD Epoch 1/200: 100%|██████████| 391/391 [00:12<00:00, 30.48it/s, Loss=7.411, Acc=7.09%]


Student KD Epoch 1: Test Accuracy = 8.41%
✅ 新的最佳准确率: 8.41%


Student KD Epoch 2/200: 100%|██████████| 391/391 [00:12<00:00, 30.39it/s, Loss=7.110, Acc=11.70%]


Student KD Epoch 2: Test Accuracy = 13.05%
✅ 新的最佳准确率: 13.05%


...

Student KD Epoch 198: Test Accuracy = 49.94%


Student KD Epoch 199/200: 100%|██████████| 391/391 [00:12<00:00, 30.77it/s, Loss=4.564, Acc=50.47%]


Student KD Epoch 199: Test Accuracy = 50.09%


Student KD Epoch 200/200: 100%|██████████| 391/391 [00:12<00:00, 30.27it/s, Loss=3.878, Acc=50.39%]


Student KD Epoch 200: Test Accuracy = 50.67%
✅ 新的最佳准确率: 50.67%
🎉 MicroResNet（知识蒸馏）训练完成! 最佳准确率: 50.67%
